In [42]:
import os
import requests
from requests.auth import HTTPBasicAuth
import json
from hashlib import md5

# @TODO get api key from environment var
# @TODO change path to production drupal path

# mailchimp api key
mailchimp_api_key = "0f8b6badf9df7076ad26563ebd6b2761-us2"
# note how the api credentials end with "us2", the data center of your accout
data_center = mailchimp_api_key.split("-")[1]
# mailchimp list id
list_id = "cdb710ff47"
# http headers
headers = {'content-type': 'application/json'}
# HTTPBasicAuth credentials
auth = HTTPBasicAuth('apikey', mailchimp_api_key)

drupal_emails = []
mailchip_emails = []

# get user emails from drupal site
def drupal_get_emails():
    # get list of emails with drush sqlq '' from a drupal site
    raw_emails = os.popen("drush -r /var/www/html/massgov.test/docroot sqlq 'select mail from users_field_data'").read()
    emails = raw_emails.split("\n")
    for email_address in emails:
        if email_address and "@" in email_address and "localhost" not in email_address:
            email = email_address.lower()
            drupal_emails.append(email)

# get members' emails from mailchimp list
def mailchimp_get_emails(data_center, list_id, count, mailchimp_api_key):
    r = requests.get("https://{}.api.mailchimp.com/3.0/lists/{}/members/?offset=0&count={}".format(data_center, list_id, count),\
                     auth=HTTPBasicAuth('apikey', '{}'.format(mailchimp_api_key)))

    # feed returned data to json
    raw_data = r.text
    response_data = json.loads(raw_data)

    # total members
    total_members = response_data["total_items"]

    # members list
    list_members = response_data["members"]

    for email in list_members:
        email = email["email_address"]
        mailchip_emails.append(email)

# get members' emails from mailchimp list
def mailchimp_delete_emails(data_center, list_id, email):
    # create md5 from email.
    # md5 value is also available via mailchip api
    # for simplicity we generate here
    m = md5()
    m.update(email.encode())
    email_hash = m.hexdigest()
    r = requests.delete("https://{}.api.mailchimp.com/3.0/lists/{}/members/{}".format(data_center, list_id, email_hash),\
                     auth=HTTPBasicAuth('apikey', '{}'.format(mailchimp_api_key)))

    if not r.status_code == 204:
        print("Failed to remove '{}' from target Mailchimp list.".format(email))
        print(r.status_code, r.request)

def mailchimp_post_emails(data_center, list_id, email):
    # api endpoint for adding users to a mailchimp list
    # note how url starts with "us2", the data center of your accout
    url = "https://{}.api.mailchimp.com/3.0/lists/{}/members".format(data_center, list_id)
    # json payload
    payload = {'email_address': email, 'status':'subscribed'}
    # payload = {'email_address':email, 'status':'subscribed', 'merge_fields':{ 'FNAME':'John', 'LNAME':'Doe' }}
    # post json data to api endpoint
    resource = requests.post(url, auth=auth, headers=headers, data=json.dumps(payload))
    # method verb cleanup
    method = str(resource.request).split(" ")[1].replace(">", "")
    # print to screen
    print(payload["email_address"], method, resource.status_code)

# get drupal users emails
drupal_get_emails()

# post drupal emails
for email in drupal_emails:
    mailchimp_post_emails(data_center, list_id, email)

#
# compare and delete email in case a user is no longer an author
#
# # get email from mailchimp
# mc_get_emails(data_center, list_id, "10", mailchimp_api_key)
#
# # delete email from mailchimp if not in drupal
# mc_delete_emails(data_center, list_id, email)
#
# # get difference between 2 listscycle
# not_in_drupal_set = set(mailchip_emails) - set(drupal_emails)
# not_in_drupal = list(not_in_drupal_set)


In [44]:
# 
# get emails list
# 
import os
from mailchimp3 import MailChimp

try:
    client = MailChimp('yriahi', '0f8b6badf9df7076ad26563ebd6b2761-us2', timeout=5.0)
    members = client.lists.members.all('cdb710ff47', get_all=True, fields="members.email_address")
    print(members)
except Exception as e:
    print(e.request)
    print(e.response)


{'members': [{'email_address': 'youssef.riahi@mass.gov'}, {'email_address': 'melissa.rossi@mass.gov'}, {'email_address': 'andrew.larrimore@mass.gov'}, {'email_address': 'linh.u.tran@mass.gov'}, {'email_address': 'carolyn.bowes@mass.gov'}, {'email_address': 'fiona.molloy@mass.gov'}, {'email_address': 'stephanie.lewis@mass.gov'}, {'email_address': 'nicola.fairhead@mass.gov'}, {'email_address': 'andrew.larrimore@state.ma.us'}, {'email_address': 'andrew.larrimore@massmail.state.ma.us'}, {'email_address': 'giles@tech-tamer.com'}, {'email_address': 'lisa.mirabile@mass.gov'}, {'email_address': 'jessica.constantine@mass.gov'}, {'email_address': 'isaac.chansky@mass.gov'}, {'email_address': 'jonathan.dallas@velir.com'}, {'email_address': 'michelle.ciccarelli@mass.gov'}, {'email_address': 'sam.mathius@mass.gov'}, {'email_address': 'meghan.davis@mass.gov'}, {'email_address': 'rob.powell@mass.gov'}, {'email_address': 'christine.flynn@mass.gov'}, {'email_address': 'wertz@palantir.net'}, {'email_addr

In [64]:
# 
# delete a specific email address
# 
email = "megan.sampson@state.ma.us"
def mc_delete_emails(data_center, list_id, email):
    # create md5 from email.
    # md5 value is also available via mailchip api
    # for simplicity we generate here
    m = md5()
    m.update(email.encode())
    email_hash = m.hexdigest()
    r = requests.delete("https://{}.api.mailchimp.com/3.0/lists/{}/members/{}".format(data_center, list_id, email_hash),\
                     auth=HTTPBasicAuth('apikey', '{}'.format(mailchimp_api_key)))
    
    if not r.status_code == 204:
        print("Failed to remove '{}' from target Mailchimp list.".format(email))
        print(r.status_code, r.request)
    else:
        print(r.status_code, r.request)
        
mc_delete_emails(data_center, list_id, email)

204 <PreparedRequest [DELETE]>


In [40]:
# 
# post an email to list
# 

import os
from mailchimp3 import MailChimp

try:
    client = MailChimp('yriahi', '0f8b6badf9df7076ad26563ebd6b2761-us2', timeout=5.0)
    client.lists.members.create('cdb710ff47', {
        'email_address': 'riahi66@yahoo.com',
        'status': 'subscribed',
        'merge_fields': {
            'FNAME': 'Salma',
            'LNAME': 'Riahi',
        },
    })
except Exception as e:
    print(e.request)
    print(e.response)


<PreparedRequest [POST]>
<Response [400]>


In [52]:
# 
# test exclusions
# 
import re

exclusions = ["localhost", "example"]
combined_exclusions = "(" + ")|(".join(exclusions) + ")"

emails = ["NULL", "user+1@localhost", "user+2@localhost",\
          "melissa.rossi@mass.gov", "andrew.larrimore@mass.gov",\
          "linh.u.tran@mass.gov", "youssef@example.com"
]
for email_address in emails:
    if email_address and "@" in email_address and not re.search(combined_exclusions, str(email_address)):
        print(email_address) 
        

melissa.rossi@mass.gov
andrew.larrimore@mass.gov
linh.u.tran@mass.gov


In [60]:
target_envir = "massgov---------.test" 
print("drush -r /var/www/html/{}/docroot sqlq 'select mail from users_field_data'".format(target_envir))


drush -r /var/www/html/massgov---------.test/docroot sqlq 'select mail from users_field_data'
